# 提取数据

由于数据源比较多，为了保持代码的可读性，将读取数据及集成部分单独放在一个模块中。

In [1]:
#用于数据的处理
import pandas as pd
import numpy as np

#用于获得上证指数数据
import tushare as ts

#忽略pandsa的警告
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getProperIpo(data):
    #取出在2014-2017年鉴上市的公司，并获得在上市日期前最后一个会计期间的数据
    #原理：通过招股开始日期，获得公司上市的年份，再取出在2014-2017来的数据，对于每一支股票按照会计年度进行排序
    #     标记每只股票最后一条数据的位置，获得上市前最后一个会计年度的数据
    #参数：data：带有会计期间、招股开始日期及股票代码、财务信息的DataFrame数据
    #返回：2014-2017件上市公司上市日期前最后一个会计期间的财务数据（DataFrame）
    
     
    data.sort_values(by=['Stkcd','date'],inplace=True,ascending=True) 
    data['stock_position']=data['Stkcd'].shift(-1)
    data['last_position']=0
    data['last_position'][data['stock_position']!=data['Stkcd']]=1
    data['last_position'][0]=0
    data2=data[data.last_position==1].drop(['stock_position','last_position','date'],axis=1)

    return data2

In [3]:
def getmarket(data,market):
    #得到最接近上市日期（上市日期之前的）的上证回报率指数
    #原理：通过循环比较特定日期与上证21天的回报率时间序列数据的日期，得到最接近上市日期前的上证回报率指数，返回上证回报率指数序列
    #参数：data:股票的上市日期序列
    #      market:上证21天的回报率指数序列
    #返回：与上市日期对应的上证21天回报率指数序列
    #注意：参数与返回值均为DataFrame格式
    
    
    market['date1']=market['date'].shift(-1)
    date=pd.to_datetime(data)
    sentiment=[]
    for time in date:
        #比较上市日期与date,date1的大小关系，得到最接近日期的回报率指数
        temp=np.array(market['21 days return'][(pd.to_datetime(market['date'])<=time)&(pd.to_datetime(market['date1'])>time)])
        sentiment.append(temp)
    s=pd.DataFrame(sentiment)
    s.columns=['Stock market sentiment']
    return s

## 上证指数21天回报率数据

In [4]:
#得到上证指数的周指数
df=ts.get_hist_data('sh', ktype='W').reset_index()

#按照时间顺序排序
df.sort_values('date',inplace=True)

#用收盘价计算21的回报率
df['close1']=df['close'].shift(-3)
df['21 days return']=(df['close1']-df['close'])/df['close']

#保存日期及回报率指数
marketReturn=df[['date','21 days return']]

In [5]:
marketReturn.head()

,date,21 days return
856,2001-06-01,-0.006091
855,2001-06-08,-0.002267
854,2001-06-15,-0.018295
853,2001-06-22,-0.020276
852,2001-06-29,-0.017317


## 汇率

- Exchdt [汇率日期] 
- Rmbusd [人民币元对美元汇率] 

In [6]:
#得到汇率数据
trdExchange=pd.read_csv("data/TRD_Exchange.csv",encoding="utf-16",sep="\t")

#对变量名进行重命名
trdExchange.rename(columns={'Exchdt':'date','Rmbusd':'US dollar buying rate'}, inplace = True)

In [7]:
trdExchange.head()

,date,US dollar buying rate
0,2010-05-10,6.8269
1,2010-05-11,6.8270
2,2010-05-12,6.8271
3,2010-05-13,6.8272
4,2010-05-14,6.8273


## 公司成立日期
- Stkcd [证券代码]
- Estbdt [公司成立日期] 

In [8]:
#得到公司的基本信息
ipoCobasic=pd.read_csv("data/IPO_Cobasic.csv",encoding="utf-16",sep="\t")

#对变量名进行重命名
ipoCobasic.rename(columns={'Estbdt':'inception date'}, inplace = True)

In [9]:
ipoCobasic.head()

,Stkcd,inception date
0,166,1996-09-16
1,333,2000-04-07
2,338,2002-12-23
3,1965,2016-08-29
4,1979,2015-06-26


## 证券发行方式、承销方式、发行价格及发行数量
- Stkcd [证券代码] 
- Ipotype [发行方式] 
- Udwwy [承销方式] - 1＝余额包销，2＝全额包销，3＝代销，4=包销，5＝其他；
- Aiprc [发行价格] - 
- Nshripo [发行数量] - 该数量为实际发行的股值。

In [10]:
#得到IPO的基本数据
ipoIpoBasic=pd.read_csv("data/IPO_Ipobasic.csv",encoding="utf-16",sep="\t").drop("T1",axis=1)

#删除无效变量：因为承销方式并无差异
ipoBasic=ipoIpoBasic.drop(['Udwwy','Aiprc'],axis=1)

#对变量名进行重命名
ipoBasic.rename(columns={'Ipotype':'Sales method','Nshripo':'number of shares sold'}, inplace = True)

In [11]:
ipoBasic.head()

,Stkcd,Sales method,number of shares sold
0,1,16,35.00
1,2,10,2800.00
2,3,19,45.20
3,4,17,1250.00
4,5,2,4433.85


## 公司上市前资产及负债情况
- Stkcd [证券代码] 
- Subbgdt [招股开始日期] - 以YYYY-MM-DD列示。
- Accper [会计年度] - 以YYYY-MM-DD列示。
- A001 [资产总计] - 资产各项目之总计
- A002 [负债合计] - 负债各项目之合计

In [12]:
#得到上市前各会计年度的资产负债情况
ipoBalanceData=pd.read_csv("data/IPO_IpoBalance.csv",encoding="utf-16",sep="\t")

#对变量名进行重命名
ipoBalanceData.rename(columns={'Accper':'date','A001':'Total assets','A002':'Total debts'}, inplace = True)

#去掉空行
ipoBalanceData.dropna(inplace=True)

#
#ipoBalanceData['Total assets']=ipoBalanceData['Total assets']/10000
#ipoBalanceData['Total debts']=ipoBalanceData['Total debts']/10000

#通过招股开始日期，获得公司上市的年份
ipoBalance=getProperIpo(ipoBalanceData)

In [13]:
ipoBalance.head()

,Stkcd,Subbgdt,Total assets,Total debts
3,100,2004-01-07,1.479043e+10,1.019994e+10
4,333,2013-09-17,9.457205e+07,5.846745e+07
8,875,2002-09-12,1.807634e+09,3.747615e+08
11,2001,2004-06-02,1.051471e+09,6.947286e+08
14,2002,2004-06-03,2.531259e+08,1.228242e+08


## 公司上市前的现金流
- Stkcd [证券代码] 
- Subbgdt [招股开始日期] - 以YYYY-MM-DD列示。
- Accper [会计年度] - 以YYYY-MM-DD列示0。
- C001 [经营活动产生的现金流量净额] 

In [14]:
#得到上市前的现金流量数据
ipoCashFlowData=pd.read_csv("data/IPO_IpoCashFlow.csv",encoding="utf-16",sep="\t")

#对变量名进行重命名
ipoCashFlowData.rename(columns={'Accper':'date','C001':'Cash flow from operations'}, inplace = True)

#去掉空行
ipoCashFlowData.dropna(inplace=True)

#
#ipoCashFlowData['Cash flow from operations']=ipoCashFlowData['Cash flow from operations']/10000

#通过招股开始日期，获得公司上市的年份
ipoCashFlow=getProperIpo(ipoCashFlowData)

In [15]:
ipoCashFlow.head()

,Stkcd,Subbgdt,Cash flow from operations
1,100,2004-01-07,-5.242400e+08
2,333,2013-09-17,3.569364e+06
4,875,2002-09-12,2.516835e+07
6,2001,2004-06-02,1.229210e+08
7,2002,2004-06-03,3.086389e+07


## 公式上市前的营业总收入、利润、及净利润
- Stkcd [证券代码] 
- Subbgdt [招股开始日期] - 以YYYY-MM-DD列示。
- Accper [会计年度] - 以YYYY-MM-DD列示。
- B0011 [营业总收入] - 企业经营过程中所有收入之和.
- B0013 [营业利润] - 与经营业务有关的利润。
- B002 [净利润] - 公司实现的净利润


In [16]:
#得到了IPO之前各会计年度的经营状况
ipoIncomeData=pd.read_csv("data/IPO_IpoIncome.csv",encoding="utf-16",sep="\t")

#对变量名进行重命名
ipoIncomeData.rename(columns={'Accper':'date','B002':'Net income','B0013':'Operating profit','B0011':'Sales'}, inplace = True)

#去掉空行
ipoIncomeData.dropna(inplace=True)

#使数值变小，更加直观
#ipoIncomeData['Sales']=ipoIncomeData['Sales']/10000
#ipoIncomeData['Net income']=ipoIncomeData['Net income']/10000
#ipoIncomeData['Operating profit']=ipoIncomeData['Operating profit']/10000

#通过招股开始日期，获得公司上市的年份
ipoIncome=getProperIpo(ipoIncomeData)

In [17]:
ipoIncome.head()

,Stkcd,Subbgdt,Sales,Operating profit,Net income
1,2140,2007-07-02,3.128633e+08,31300415.10,27361736.36
2,2141,2007-07-11,3.089272e+08,13329091.71,11732536.99
3,2143,2007-07-11,8.722297e+08,20385934.66,20835152.20
4,2144,2007-07-20,1.638524e+08,14003759.54,12243657.80
5,2145,2007-07-20,1.780167e+08,-4323499.07,1867282.77


## 上市首日的个股回报率
- Stkcd [证券代码] 
- Listdt [上市日期] 
- Retnfstd [上市首日的个股回报率] - （股票在首日的收盘价-股票n的招股价格）-1
- Retnadfstd （1+上市首日的个股回报率）/（1+上市首日的市场回报率）-1


In [18]:
#上市首日市场表现
ipoIpoday=pd.read_csv("data/IPO_Ipoday.csv",encoding="utf-16",sep="\t")
ipoIpoday['Offer price']=ipoIpoday['Clsprc']/(ipoIpoday['Retnfstd']+1)
ipoIpoday=ipoIpoday.drop(['Clsprc','Retnfstd'],axis=1)
#对变量名进行重命名
ipoIpoday.rename(columns={'Retnadfstd':'One day excess return','Listdt':'date'}, inplace = True)

In [19]:
ipoIpoday.head()

,Stkcd,date,One day excess return,Offer price
0,166,2015-01-26,NaN,4.860000
1,333,2013-09-18,NaN,44.560017
2,1965,2017-12-25,NaN,8.180000
3,1979,2015-12-30,NaN,23.600009
4,2396,2010-06-23,0.322877,23.200000


# 合并数据集

In [20]:
def getFirmAge(date1,date2):
    #得到两个日期之间相差的天数
    #参数：date1、date2，均为pandas的series序列
    #返回：天数的pandas-series序列
    date11=pd.to_datetime(date1)
    date21=pd.to_datetime(date2)
    days=(date11-date21).apply(lambda x:x.days)/365
    return days

In [21]:
#以负债表中的公司为准，合并IPO各数据
IPO_a=pd.merge(ipoBalance,ipoCashFlow, how='inner', on=['Stkcd','Subbgdt'])  
IPO_b=pd.merge(IPO_a,ipoIncome,on=['Stkcd','Subbgdt'],how="inner")  
IPO_c=pd.merge(IPO_b,ipoIpoday,on=['Stkcd'],how="inner")
IPO_d=pd.merge(IPO_c,ipoBasic,on=['Stkcd'],how="inner")
IPO_e=pd.merge(IPO_d,ipoCobasic,on=['Stkcd'],how="inner")
#合并汇率数据
IPO_f=pd.merge(IPO_e,trdExchange,on='date',how="inner")

In [23]:
#计算ROA1
IPO_f['ROA1']=IPO_f['Net income']/IPO_f['Total assets']

#计算ROA2
IPO_f['ROA2']=IPO_f['Operating profit']/IPO_f['Total assets']

#计算资金周转率
IPO_f['Total assets turnover rate']=IPO_f['Sales']/IPO_f['Total assets']

#计算资产负债比
IPO_f['Debt ratio']=1-(IPO_f['Total debts']/IPO_f['Total assets'])

#计算公司成立年限
IPO_f['Firm age']=getFirmAge(IPO_f['date'],IPO_f['inception date'])

#计算发行收益
IPO_f['Issue proceeds(USD)']=IPO_f['number of shares sold']*IPO_f['Offer price']/IPO_f['US dollar buying rate']

#得到新股发行的年份
IPO_f['year']=pd.to_datetime(IPO_f['date']).dt.year

In [24]:
#加入对应的上证21天回报率指数
IPO_f['Stock market sentiment']=getmarket(IPO_f['Subbgdt'],marketReturn)


In [25]:
#去掉空行
IPO_f.dropna(inplace=True)

#取得2014到2017间的数据
IPO_f=IPO_f[(IPO_f['year']>=2014)&(IPO_f['year']<=2017)]

#对index重新进行排序
IPO_g=IPO_f.reset_index(drop=True) 

In [26]:
#得到最终的数据集
IPO=IPO_g.drop(['date','inception date','Subbgdt','US dollar buying rate'],axis=1)

In [27]:
IPO.head()

,Stkcd,Total assets,Total debts,Cash flow from operations,Sales,Operating profit,Net income,One day excess return,Offer price,Sales method,number of shares sold,ROA1,ROA2,Total assets turnover rate,Debt ratio,Firm age,Issue proceeds(USD),year,Stock market sentiment
0,2705,3.390104e+09,2.068954e+09,5.592463e+08,5.041712e+09,2.447157e+08,1.922726e+08,0.467994,10.500000,"8,12",7600.0000,0.056716,0.072185,1.487185,0.389708,18.126027,13067.827823,2014,-0.022301
1,2706,6.125454e+08,2.356095e+08,6.992491e+07,6.838492e+08,8.796354e+07,8.105681e+07,0.425468,19.099998,"8,12",2154.0000,0.132328,0.143603,1.116406,0.615360,7.293151,6737.201547,2014,0.009827
2,2708,7.845328e+08,2.831599e+08,3.287824e+07,5.805561e+08,6.194554e+07,5.317030e+07,0.420506,11.880000,"8,12",3320.0000,0.067773,0.078959,0.740002,0.639072,2.986301,6458.847594,2014,0.009827
3,300357,2.635215e+08,1.223968e+07,5.558691e+07,1.936098e+08,8.163514e+07,6.991001e+07,0.469495,20.050005,"8,12",2525.0000,0.265291,0.309785,0.734702,0.953553,11.347945,8290.417208,2014,-0.022301
4,300358,1.076322e+09,6.208643e+08,1.388938e+08,7.972057e+08,1.400280e+08,1.351335e+08,0.446298,40.000000,"8,12",1824.9813,0.125551,0.130099,0.740676,0.423161,11.210959,11954.156486,2014,-0.022301


In [28]:
#由于不是8,12及12销售方式的用户比较少.
IPO['Sales method'].value_counts()

8,12       851
12         154
7            1
18,8,12      1
Name: Sales method, dtype: int64

In [29]:
#查看它们所在的行
IPO['Sales method'][(IPO['Sales method']=='18,8,12')|(IPO['Sales method']=='7')]

35     18,8,12
765          7
Name: Sales method, dtype: object

In [30]:
#构建新的销售方式变量，在采用了12的销售方式的基础上还采用了8的，取值为1，反之为0
IPO['Sales method1']=1
IPO['Sales method1'][IPO['Sales method']=="12"]=0
IPO['Sales method1'].value_counts()

#删除采用数量较少的两种销售方式
ipo1=IPO.drop([519,586],axis=0)
ipo1.dropna(axis=0,how='any') 

#重命名
ipo=ipo1.drop(['Sales method'],axis=1)
ipo.rename(columns={'Sales method1':'Sales method'}, inplace = True)

In [31]:
ipo.to_csv("IPO.csv",index=False)